In [1]:
import cv2
from pathlib import Path
import glob
import numpy as np
from matplotlib import pyplot as plt
import os

In [2]:
cv2.__version__

'3.4.4'

In [3]:
# path to the dataset containing subdirectories with images
dataset_path_str = '/disks/data/datasets/selfie_project/selfie_dataset_versions/100k_copy/'

# sub directory where the to-be inspected reside
class_label = 'selfie'

# flag deciding whether to show the selected image in a separate window
show_selected_images = False

# image montage parameters
MONTAGE_COL_COUNT = 5
MONTAGE_ROW_COUNT = 5

# size of individual images in the montage (width, height)
IMSIZE = (150,150)

In [4]:
# new directories under 'dataset_path' where filtered images will be moved to 
moved_img_class_no = 'no_' + class_label
moved_img_class_yes = 'yes_' + class_label

In [5]:
dataset_path = Path(dataset_path_str)
img_path = dataset_path/class_label

# full paths of directories where selected and unselected images will be moved to
newdir_yes = dataset_path/moved_img_class_yes
newdir_no = dataset_path/moved_img_class_no

# create directories for the moved images
os.makedirs(newdir_yes, exist_ok = True)
os.makedirs(newdir_no, exist_ok = True)


In [6]:
def create_montage(imglist, montage_shape, img_sz):
    '''
    creates montage from a list of image filename, montage shape, and per-image size.
    
    params:
        imglist (str): list of filepaths of the images
        montage_shape (tuple): contains the dimensions (row_count, col_count) of the montage
        img_sz (tuple): contains the dimensions (height, width [, depth) of each image in the montage
    
    returns:
        numpy ndarray containing the montage of images.
    '''
    
    # create an empty canvas for the montage
    canvas = np.zeros( (img_sz[0]*montage_shape[0], img_sz[1]*montage_shape[1], 3), np.uint8)
    
    # fill-up the montage row-by-row
    for icol in range(montage_shape[1]):
        for irow in range(montage_shape[0]):
            # x-y coords to linear index 
            img_idx = irow*montage_shape[1] + icol
            
            # read and resize the image; on failure use a red placeholder
            try:
                img = cv2.resize(cv2.imread(imglist[img_idx]),img_sz[:2])
            except Exception as e:
                img = np.zeros((*img_sz[:2],3), np.uint8)
                img[::-1] = 255
                print('file read error',imglist[img_idx])
            
            # populate the canvas with the image
            canvas[irow*img_sz[0]: (irow+1)*img_sz[0],
                  icol*img_sz[1]: (icol+1)*img_sz[1],:] = img
            
    # return the montage        
    return canvas

In [7]:
# shape and size of the grid
IMGRID_SHAPE = (MONTAGE_COL_COUNT, MONTAGE_ROW_COUNT)
IMGRID_SIZE = np.prod(IMGRID_SHAPE)

In [8]:
def process_selected_imgs(selected_imgs, images):
    '''
    Move selected and unselected images to other, separate directories
    
    params:
        selected_imgs (list): list of selected images' paths
        images (list): list of all images' paths from the currently shown montage
    
    returns:
        Nothing
    '''
    
    # convert list to sets to eliminate duplicates
    selected_imgs = set(selected_imgs)
    # find the list of unselected images
    unselected_imgs = list(set(images)-set(selected_imgs))
    
    # generate paths of the moved (selected and unselected) images
    newpaths_yes = [os.path.join(newdir_yes, fname) for fname in [ fname for path, fname in [os.path.split(filepath) for filepath in unselected_imgs ] ] ]
    newpaths_no = [os.path.join(newdir_no, fname) for fname in [ fname for path, fname in [os.path.split(filepath) for filepath in selected_imgs ] ] ]
    
    # move the selected and unselected images
    [os.rename( fr_file, to_file) for fr_file, to_file in zip(unselected_imgs, newpaths_yes)]
    [os.rename( fr_file, to_file) for fr_file, to_file in zip(selected_imgs, newpaths_no)]
    

In [9]:
# mouse event handler: to highlight selected image and add to the leave-out list
def highlight_img(event, x, y, *kwargs):

    global im_fnames_montage
    global image_montage
    global selected_imgs_coords
    global selected_imgs_names
    global IMSIZE
    global IMGRID_SHAPE
    
    # get montage image-space coordinates of the selected image
    def get_rectangle_corners(top_left, im_size, line_width):
        return ( (top_left[0]*im_size[0] + line_width//2, top_left[1]*im_size[1] + line_width//2),
                 ((top_left[0]+1)*im_size[0] - line_width//2, (top_left[1]+1)*im_size[1]- line_width//2) )

    def draw_border(im_coords, line_width, color):
        # montage image-space coordinates
        corners = get_rectangle_corners(im_coords, IMSIZE, line_width)

        # index of the selected image in the montage
        im_idx = im_coords[1]*IMGRID_SHAPE[0] + im_coords[0]
        
        # the file name of the selected image
        im_name = im_fnames_montage[im_idx]
        #print(im_idx, im_name)
        
        # draw a rectangular border around the selected image
        cv2.rectangle(image_montage, corners[0], corners[1],color, line_width )            
        
        return im_name
    
    # width of the rectangle border to be drawn around the selected image
    line_width = 6

    # draw border around the select image and add it to the list
    if event == cv2.EVENT_LBUTTONUP:
        
        # grid-space (zero-indexed) coordinates of the selected image
        im_coords = (x//IMSIZE[0], y//IMSIZE[1])
        
        selected_imgs_coords.append(im_coords)
        
        im_name = draw_border(im_coords, line_width,  (0,0,255))
        
        selected_imgs_names.append(im_name)
        
        # show the modified montage
        cv2.imshow("Montages", image_montage)
        
        # show the selected image (optional) - uncomment if needed
        if(show_selected_images):
            cv2.namedWindow("Selected")
            cv2.imshow("Selected", cv2.imread(im_name))
            cv2.waitKey(0)
            cv2.destroyWindow("Selected")
     
     # undo the last selection with a right-click
    elif event == cv2.EVENT_RBUTTONUP:  
        if len(selected_imgs_names) > 0:
            
            # remove the image file name from no-move queue
            im_name = selected_imgs_names.pop()
            
            # retrieve the coordinate of the previous selection
            latest_img_coords = selected_imgs_coords.pop()

            # draw a green border around the de-selected image
            draw_border(latest_img_coords, line_width, (0,255,0))
            
            # show the modified montage
            cv2.imshow("Montages", image_montage)

In [12]:
# read in list of image files
img_files = glob.glob(str(img_path/'*.jpg'))

# generate indices for the list of all images        
im_indices = list(range(len(img_files)))

# flag indicating to quite labeling 
do_quit = False

# show 'imgrid_size' images at a time
for i in im_indices[::IMGRID_SIZE]:

    # quit labeling
    if do_quit:
        break

    # indices of the images in current montage    
    imgs_idx_montage = slice(i, min(i+IMGRID_SIZE, len(im_indices)) )

    # filenames of the images in the montage
    im_fnames_montage = img_files[imgs_idx_montage]
    
    # list containing the filenames of the selected images
    selected_imgs_names = []
    selected_imgs_coords = []
    
    # list containing images for the montage
    images = []

    # make a list of imgs (for montage function)
    for img_fname in im_fnames_montage:
        img = cv2.imread(img_fname)
        images.append(img)
    
    # build the montage image        
    #image_montage = build_montages(images, IMSIZE, IMGRID_SHAPE )[0]
    image_montage = create_montage(im_fnames_montage, IMGRID_SHAPE[::-1], IMSIZE[::-1])
    
    # make the image window
    cv2.namedWindow("Montages")
    
    # move the image window so that it doesn't get clipped
    cv2.moveWindow("Montages", 50,50);
    
    # attach mouse event handler to the image window
    cv2.setMouseCallback("Montages", highlight_img)

    # display the image and wait for a keypress
    cv2.imshow("Montages", image_montage)
 
    # read the keystroke
    key = cv2.waitKey(0) & 0xFF
     
    # exit the loop
    if key == ord('q'):
        do_quit = True
        continue
        
    # process selected (and unselected) images
    process_selected_imgs(selected_imgs_names, im_fnames_montage)
    
    
# destroy all opencv windows
cv2.destroyAllWindows()
# wait for at most one millisecond 
key = cv2.waitKey(1) & 0xFF
